In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
import random
from datetime import datetime
import time

from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=100000000
%config NotebookApp.rate_limit_window=20.0

In [2]:
df_1 = pd.read_csv('crime.csv')
df_1.head(20)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802


In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
categories = df_1['Category'].unique().tolist()
descripts = df_1['Descript'].unique().tolist()
daysOfWeek = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
districts = df_1['PdDistrict'].unique().tolist()
resolutions = df_1['Resolution'].unique().tolist()

In [5]:
def randIDX(df, category, num):
    filtered_df = df.loc[df.loc[:,'Category'] == category]
    indices = filtered_df.index.tolist()
    random_indices = random.sample(indices, min(num, len(indices)))
    return random_indices

In [7]:
random.seed(7)

trainNum = 10000
testNum = 3500

trainIDX = []
testIDX = []

staticIDX = []
for i in range(len(categories)):
    staticIDX.append(randIDX(df_1, categories[i], 5))

while len(trainIDX) < trainNum:
    for i in range(len(staticIDX)):
        for j in range(len(staticIDX[i])):
            trainIDX.append(staticIDX[i][j])
    num = random.randint(0, len(df_1)-1)
    if not num in trainIDX:
        trainIDX.append(num)
        
while len(testIDX) < testNum:
    num = random.randint(0, len(df_1)-1)
    if not num in testIDX and not num in trainIDX:
        testIDX.append(num)

testDF = df_1.iloc[testIDX].reset_index(drop=True)
trainDF = df_1.iloc[trainIDX].reset_index(drop=True)

In [24]:
def findWords(df, category, current, total):
    locDF = df.loc[df.loc[:,'Category'] == category]
    
    currIteration = 1
    allWords = []
    for descript in locDF['Descript']:
        words = descript.split()
        for word in words:
            allWords.append(word)
        percentageDone = round((((currIteration/len(locDF))*33.33333)*(1/total))+(((current-1)/total)*100),2)
        clear_output(wait=True)
        print(str(percentageDone) + '% done, category: ' + str(current) + '/' + str(total))
        time.sleep(0.001)
        currIteration += 1

    dictionary = {'category':[], 'topWord':[], 'count':[], 'proportionOfAllWords':[]}
    word_counts = pd.DataFrame(dictionary)
    
    currIteration = 1

    for word in allWords:
        if word in word_counts['topWord'].tolist():
            index = word_counts[word_counts['topWord'] == word].index[0]
            word_counts.loc[index, 'count'] += 1
        else:
            new_word = pd.DataFrame({'category':[category], 'topWord':[word], 'count':[1]})
            word_counts = pd.concat([word_counts, new_word], ignore_index=True)
        percentageDone = round(((((currIteration/len(allWords))*33.33333)+33.33333)*(1/total))+(((current-1)/total)*100),2)
        clear_output(wait=True)
        print(str(percentageDone) + '% done, category: ' + str(current) + '/' + str(total))
        time.sleep(0.001)
        currIteration += 1
    
    for j in range(len(word_counts)):
        word_counts.loc[j, 'proportionOfAllWords'] = word_counts.loc[j, 'count']/len(allWords)
        percentageDone = round((((((j+1)/len(word_counts))*33.33333)+66.66667)*(1/total))+(((current-1)/total)*100),2)
        time.sleep(0.001)
        clear_output(wait=True)
        print(str(percentageDone) + '% done, category: ' + str(current) + '/' + str(total))
         
    word_counts = word_counts.sort_values(by='count', ascending=False).reset_index(drop=True)
    word_counts = word_counts.drop('count', axis=1).reset_index(drop=True)  
#     return word_counts.reset_index(drop=True).loc[[0]]
    if len(word_counts) > 2:
        return {'category': [word_counts.loc[0, 'category']],
        'topWord': [word_counts.loc[0, 'topWord']], 
        'secondWord': [word_counts.loc[1, 'topWord']],
        'thirdWord': [word_counts.loc[2, 'topWord']],
        'proportionOfAllWords': [word_counts.loc[0, 'proportionOfAllWords']]}
    elif len(word_counts) > 1:
        return {'category': [word_counts.loc[0, 'category']],
        'topWord': [word_counts.loc[0, 'topWord']], 
        'secondWord': [word_counts.loc[1, 'topWord']],
        'thirdWord': ['null'],
        'proportionOfAllWords': [word_counts.loc[0, 'proportionOfAllWords']]}
    else:
        return {'category': [word_counts.loc[0, 'category']],
        'topWord': [word_counts.loc[0, 'topWord']], 
        'secondWord': ['null'],
        'thirdWord': ['null'],
        'proportionOfAllWords': [word_counts.loc[0, 'proportionOfAllWords']]}

In [25]:
df = {'category':[], 'topWord':[], 'secondWord':[], 'thirdWord':[], 'proportionOfAllWords':[]}
df = pd.DataFrame(df)

for i in range(len(categories)):
    row = pd.DataFrame(findWords(trainDF, categories[i], (i+1), len(categories)))
    df = pd.concat([df, row], ignore_index=True)

df.head()

100.0% done, category: 39/39


,category,topWord,secondWord,thirdWord,proportionOfAllWords
0,WARRANTS,WARRANT,ARREST,ENROUTE,0.306569
1,OTHER OFFENSES,VIOLATION,OF,MUNICIPAL,0.133959
2,LARCENY/THEFT,THEFT,GRAND,FROM,0.208366
3,VEHICLE THEFT,STOLEN,AUTOMOBILE,"VEHICLE,",0.307692
4,VANDALISM,MALICIOUS,"MISCHIEF,",VANDALISM,0.250237


In [26]:
df

,category,topWord,secondWord,thirdWord,proportionOfAllWords
0,WARRANTS,WARRANT,ARREST,ENROUTE,0.306569
1,OTHER OFFENSES,VIOLATION,OF,MUNICIPAL,0.133959
2,LARCENY/THEFT,THEFT,GRAND,FROM,0.208366
3,VEHICLE THEFT,STOLEN,AUTOMOBILE,"VEHICLE,",0.307692
4,VANDALISM,MALICIOUS,"MISCHIEF,",VANDALISM,0.250237
5,NON-CRIMINAL,AIDED,PROPERTY,LOST,0.154179
6,ROBBERY,ROBBERY,ON,THE,0.147622
7,ASSAULT,BATTERY,THREATS,AGAINST,0.942446
8,WEAPON LAWS,OF,POSSESSION,CARRYING,0.106703
9,BURGLARY,ENTRY,UNLAWFUL,BURGLARY,0.217247


In [23]:
len(df['topWord'].unique())

35